In [ ]:
import numpy as np
from csv import reader

In [ ]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    return (actual == predicted).sum() / actual.shape[0] * 100.0

In [3]:
def rmse_metric(actual, predicted):
    return np.sqrt(np.mean((predicted-actual)**2))

In [4]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    # random shuffle before sending 
    np.random.shuffle(dataset)
    
    return np.array_split(dataset, n_folds)

In [5]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, metric, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for i in range(n_folds):
        test = folds[i]
        train = np.concatenate([folds[j] for j in range(n_folds) if j != i])
        
        predicted = algorithm(train, test, *args)
        actual = test[:, -1]
        
        result = metric(actual, predicted)
        scores.append(result)
    
    return scores

# Decision tree

Calculate the Gini index for a split dataset

In [6]:
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        
        # avoid divide by zero
        if size == 0: continue
        score = 0.0
        
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    
    return gini

In [7]:
# test Gini values
assert gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]) == 0.5
assert gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]) == 0.0

Split a dataset based on an attribute and an attribute value

In [8]:
def create_split(index, value, dataset):
    return [dataset[dataset[:, index] < value], dataset[dataset[:, index] >= value]]

In [9]:
# Test getting the best split
dataset = np.array([[2.771244718,1.784783929,0],
  [1.728571309,1.169761413,0],
  [3.678319846,2.81281357,0],
  [3.961043357,2.61995032,0],
  [2.999208922,2.209014212,0],
  [7.497545867,3.162953546,1],
  [9.00220326,3.339047188,1],
  [7.444542326,0.476683375,1],
  [10.12493903,3.234550982,1],
  [6.642287351,3.319983761,1]])
index = 1
value = 2.7
left, right = create_split(index, value, dataset)

Select the best split point for a dataset

In [10]:
def get_split(dataset):
    class_values = np.unique(dataset[:, -1])
    b_index, b_value, b_score, b_groups = float("inf"), float("inf"), float("inf"), None
    
    for index in range(dataset.shape[1]-1):
        for row in dataset:
            groups = create_split(index, row[index], dataset)
            
            gini = gini_index(groups, class_values)
            
#             print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    
    return {'index': b_index, 'value': b_value, 'groups': b_groups}


In [11]:
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))


Split: [X1 < 6.642]


In [12]:
# Create a terminal node value
def to_terminal(group):
    u, c = np.unique(group[:, -1], return_counts=True)
    return u[c == c.max()][0]


In [13]:
# Create a terminal node value
def check_unique(arr):
    return np.unique(arr[:, -1]).shape[0] == 1


In [14]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    
    # check for a no split
    if not left.shape[0] or not right.shape[0]:
        node['left'] = node['right'] = to_terminal(np.concatenate((left, right)))
        return
    
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size or check_unique(left):
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    
    # process right child
    if len(right) <= min_size or check_unique(right):
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        

In [15]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root


In [16]:
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))
        

In [17]:
tree = build_tree(dataset, 3, 1)
print_tree(tree)


[X1 < 6.642]
 [0.0]
 [1.0]


In [18]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return np.array(dataset, dtype=np.float)

In [19]:
# load and prepare data
filename = 'data/data_banknote_authentication.csv'
dataset = load_csv(filename)

In [20]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [21]:
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    
    return np.array(predictions)

In [22]:
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, accuracy_metric, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [96.36363636363636, 97.45454545454545, 96.35036496350365, 93.7956204379562, 97.08029197080292]
Mean Accuracy: 96.209%
